In [7]:
# versão 3
import simpy
import random
import numpy as np


class icusim():
    def __init__(self, env) -> None:
        
        self.LEITOS_TOTAL = 3
        self.MEDIA_TEMPO_INTERNACAO = 10 #DIAS
        self.DESVIO_PADRAO_TEMPO_INTERNACAO = 2 # DIAS
        self.MEDIA_TEMPO_CHAGADA_PACIENTE =  6 #HORAS
        self.DESVIO_PADRAO_TEMPO_CHAGADA_PACIENTE = 1 # HORAS
        
        
        self.env = env
        self.pacientes_solicitacao = 0
        self.pacientes_internados = 0
        self.pacientes_atendidos = 0
        
        self.leitos = simpy.Resource(env, capacity=self.LEITOS_TOTAL)
        
    def paciente(self, nome, tempo_internacao):
        print(f'Paciente {nome} solicitando UTI em {self.env.now}')
        leito_ocupado = self.leitos.request()
        yield leito_ocupado
        print(f'Paciente {nome} Entrada na UTI em {self.env.now}' )
        self.pacientes_internados += 1
        yield self.env.timeout(tempo_internacao)
        print(f'Paciente {nome} Saída da UTI em {self.env.now}' )
        self.leitos.release(leito_ocupado)
        self.pacientes_atendidos += 1
        self.pacientes_internados -= 1

    def cria_paciente(self):
        paciente_nome = 0
        tempo_internacao = self.tempo_internacao()*24 # converte para horas 
        while True:
            # nesta linha tem o tempo de chegada de um novo paciente
            yield env.timeout(self.tempo_chegada_paciente())
            self.pacientes_solicitacao += 1
            print(f'Paciente {paciente_nome} criado em {env.now}')
            self.env.process(self.paciente(paciente_nome, tempo_internacao))
            paciente_nome += 1

    def tempo_internacao(self):
        return np.random.normal(self.MEDIA_TEMPO_INTERNACAO, self.DESVIO_PADRAO_TEMPO_INTERNACAO)

    def tempo_chegada_paciente(self):
        # impede tempos negativos
        resposta = -1
        while resposta < 0:
            resposta = np.random.normal(self.MEDIA_TEMPO_CHAGADA_PACIENTE, self.DESVIO_PADRAO_TEMPO_CHAGADA_PACIENTE)
        return resposta


# unidade temporal é HORAS
            
env = simpy.Environment()
hospital = icusim(env)
env.process(hospital.cria_paciente())
env.run(until=8*24)

print(f'Pacientes solicitados: {hospital.pacientes_solicitacao}')
print(f'Pacientes internados: {hospital.pacientes_internados}')
print(f'Pacientes atendidos: {hospital.pacientes_atendidos}')


Paciente 0 criado em 5.459878761620755
Paciente 0 solicitando UTI em 5.459878761620755
Paciente 0 Entrada na UTI em 5.459878761620755
Paciente 1 criado em 11.653002404761857
Paciente 1 solicitando UTI em 11.653002404761857
Paciente 1 Entrada na UTI em 11.653002404761857
Paciente 2 criado em 19.015334854836016
Paciente 2 solicitando UTI em 19.015334854836016
Paciente 2 Entrada na UTI em 19.015334854836016
Paciente 3 criado em 24.56515150237415
Paciente 3 solicitando UTI em 24.56515150237415
Paciente 4 criado em 29.385516751587893
Paciente 4 solicitando UTI em 29.385516751587893
Paciente 5 criado em 35.70173852975836
Paciente 5 solicitando UTI em 35.70173852975836
Paciente 6 criado em 41.148733367281444
Paciente 6 solicitando UTI em 41.148733367281444
Paciente 7 criado em 48.084007295173215
Paciente 7 solicitando UTI em 48.084007295173215
Paciente 8 criado em 55.48265290519084
Paciente 8 solicitando UTI em 55.48265290519084
Paciente 9 criado em 61.273518452619115
Paciente 9 solicitando U